# Indexing the PM2.5 Data Set

## Table of Contents:

1. [Extract station data](#extract_stations)
2. [Index measuring data](#index_measureing_data)
3. [Correct frequency](#correct_frequency)
4. [How to use index](#how_to_use_index)
    1. [Query to index](#query_to_index) 
    2. [Coverage of data](#coverage_of_data)

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parents[0]
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

import numpy as np
import pandas as pd
from src.config import DATA_URLS_FILE, INFO_URLS_FILE, RAW_DIR, METADATA_DIR, PROCESSED_DIR, \
STATIONS_RAW_CSV, STATIONS_CSV, INDEX_CSV, CHECKED_FREQUENCY

from src.data.index_data import *
from src.data.index_query import get_years_for_site
from src.visualization.coverage_maps import visualize_coverage_by_site_and_year


## 1. Extract station data <a name="extract_stations"></a>

We will extract all NAPS site ID and some information, like land use and scale, for later use. These information will be saved in a CSV file, `/data/metadata/stations.csv`.

<div class="alert alert-block alert-info">
<b>For verification:</b> Check the first and last NAPS stations listed in the original CSV. The following code will print out the first and last three stations. If the extracted first and last stations do not agree with those on the origianl CSV, you may need to update the code.
</div>

In [ ]:
extract_stations(STATIONS_RAW_CSV, STATIONS_CSV, METADATA_DIR)


## 2. Index measuring data<a name="index_measureing_data"></a>

Next, we will create a CSV file that contains a list of the attributes of the data files. This project will particularly be interested in PM2.5 speciation data. The presence and absence of a particular data will be determined by checking the data set with with the following rules depending on a year.

| analyte type | 2003 - 2009 | 2010 - 2019 |
| ---- | ------- | ------- |
| Near Total metal | file named `*_ICPMS.XLS` | worksheet named `Metals_ICPMS (Near-Total)` |
| Water-soluble metal | file named `*_WICPMS.XLS` | worksheet named `Metals_ICPMS (Water-Soluble)` |
| ions | file named `*_IC.XLS` | worksheet named `Ions-Spec_IC` |

The result will be stored in an index CSV file, which structure is the following.

| year | site_id | analyte | analyte_type | instrument | frequency |
| ---- | ------- | ------- | ---------- | ---------- | --------- |
| measurement year | NAPS site ID | full name of the analyte | 'NT' (near total) or 'WS' (water-soluble) for ICPMS; 'total' for IC | ICPMS or IC | measurement frequency (day) |

If the data set contains at least one entry for an analyte, a row will be created in this index file, even if the reported value was error measurement. The actual number of non-error measurement and ther percentage of that will be checked later.

The differentiation of NT and WS will be later used to extract PM2.5 data. Because Near Total data is from Sampler #1 and Water-soluble data is from Sampler #2, we have to match a correct PM2.5 dataset with a correponding metal dataset.

<div class="alert alert-block alert-info">
<b>For maintenance:</b> Modify unzipped_dir_for function in napsanalyzer.py appropriately so that it will return a suitable directory path when you add the data set of newer year(s) and/or when a provided zip file changes its directory strcuture.

Also, if the use of Samplers is changed in the future, we will have to change the code for extraction.
</div>

In [ ]:
index_dataset_attributes(DATA_URLS_FILE, INDEX_CSV, RAW_DIR)


**Example of the use of index CSV**

The following code will show the sites with IC-measured data but no ICPMS-measured data and the number of such sites.


In [ ]:
index_df = pd.read_csv(INDEX_CSV)

# filter for 'IC' and 'ICPMS'
ic_sites = index_df[index_df['instrument'] == 'IC']['site_id'].unique()
icpms_sites = index_df[index_df['instrument'] == 'ICPMS']['site_id'].unique()

# find site_ids with 'IC' but not with 'ICPMS'
sites_with_ic_not_icpms = set(ic_sites) - set(icpms_sites)

# convert to list and count
sites_with_ic_not_icpms_list = list(sites_with_ic_not_icpms)
count_sites_with_ic_not_icpms = len(sites_with_ic_not_icpms_list)

# display the site_ids and count
print("Site IDs with data measured by IC but not ICPMS:", sites_with_ic_not_icpms_list)
print("Number of such Site IDs (without ICPMS-measured data):", count_sites_with_ic_not_icpms)

## 3. Correct frequency<a name="correct_frequency"></a>

Some of the calculated frequency were not accurate because sampling dates are sometimes very spaced and the current code does not follow it. To deal with such a case, the frequency was intentionally assigned as 100 for an alert to be manually corrected. Also, some files contain consequent sampling date, which is not correct; the frequency should be **one for three day** or **six day**. So, the following function will manually correct the wrongly detected frequency in the index CSV.

First, this code will show you how many entries may have a wrong frequency.

In [ ]:
print(f'Total {len(index_df)} entries in the index CSV')

filtered_df = index_df[(index_df['frequency'] != 3) & (index_df['frequency'] != 6)]

grouped_df = filtered_df.groupby(['year', 'site_id', 'analyte_type', 'instrument', 'frequency']).size().reset_index(name='count')
print('The frequency for ' + str(len(grouped_df)) + ' data sets (' + str(len(filtered_df)) + ' rows in the index CSV) may be wrong:')
display(grouped_df.head(3))
display(grouped_df.tail(3))

The following function will apply the manually-checked data (in CSV) and will save it.


In [ ]:
index_df = apply_manually_checked_frequency(index_df, CHECKED_FREQUENCY, INDEX_CSV)

# show the result
print(f'Total {len(index_df)} entries in the index CSV')

filtered_df = index_df[(index_df['frequency'] != 3) & (index_df['frequency'] != 6)]
display_columns = ['year', 'site_id', 'analyte_type', 'instrument', 'frequency']

grouped_df = filtered_df.groupby([
    'year', 'site_id', 'analyte_type', 'instrument', 'frequency']).size().reset_index(name='count')

print('The frequency for ' + str(len(grouped_df)) + 
      ' data sets (' + str(len(filtered_df)) + ' rows in the index CSV) may be wrong:')

display(grouped_df)


The NAPS program reported only one or two measurement for the following data files. 

| year | site_id | analyte_type | instrument |
| ---- | ------- | ------- | ---------- |
| 2007 | 70301 | WS | ICPMS |
| 2016 | 129302 | total | IC |
| 2017 | 60610 | total | IC |

So, the following function will drop these data from the index CSV.

In [ ]:
# drop some entries
index_df = drop_entries_with_too_few_measurements(index_df, INDEX_CSV)

# show the result 
filtered_df = index_df[(index_df['frequency'] != 3) & (index_df['frequency'] != 6)]
grouped_df = filtered_df.groupby([
    'year', 'site_id', 'analyte_type', 'instrument', 'frequency']).size().reset_index(name='count')
print('The frequency for ' + str(len(grouped_df)) + 
      ' data sets (' + str(len(filtered_df)) + ' rows in the index CSV) may be wrong:')
display(grouped_df)

Now, no row with the frequency other than three or six remains.

Also, we need to correct the frequency for some data files. The frequency changed from 6 to 3 in the early phase of the year for these datafiles. So, the following function will overwrite the information and save it to the CSV file.

In [ ]:
# update some entries and save it 
index_df = update_index_with_major_frequency(index_df, INDEX_CSV)

# check the number of entries
print(f'Total {len(index_df)} entries in the index CSV')


## 4. How to use index<a name="how_to_use_index"></a>

### 4.1. Query to index<a name='query_to_index'></a>

To find out which years' data are available for a particular NAPS site, analyte, and analyte type, you can filter the DataFrame by the corresponding `site_id`, `analyte`, and `analyte_type`, and then get the unique list of years for that `site_id`. Here's how you can do it:


In [ ]:
# Replace with the site_id you want to check
specific_site_id = 100119

# Replace with the analyte you want to check
specific_analyte = 'lead'

# Replace with the analyte you want to check ('NT' or 'WS' for ICPMS data, 'IC' for ions)
specific_analyte_type = 'NT'  

unique_years_list = get_years_for_site(specific_site_id, specific_analyte, specific_analyte_type)

print(f"Unique years available for {specific_analyte_type}-{specific_analyte} \
at site {specific_site_id}:\n {unique_years_list}")


### 4.2. Coverage of data<a name="coverage_of_data"></a>

You can also take a look at the coverage of the data at each site. This coverage will be created based on the presence of, at least, three measurements, refering to the index CSV. The following code will show a coverage table for ICPMS-measured data without specifying an analyte.

In [ ]:
visualize_coverage_by_site_and_year()


The above table shows that 28 sites have PM2.5 data measured by ICP-MS. Let's check the difference between this number and the number of the stations that currently provide the ICP-MS measured data.

In [ ]:
# check the number of stations that currently have ICP-MS measurements
stations = pd.read_csv(STATIONS_CSV)
icpms_stations = stations[stations[
    'PM2.5_Speciation'] == 'X'].loc[:, ['site_id', 'station_name', 'Start_Year', 'End_Year']].reset_index()

print(f'PM2.5 Speciation data is currently measured at {len(icpms_stations)} \
sites, according to StationsNAPS-StationsSNPA.csv.')

# check the number of stations that have ICP-MS measurements based on parsing by code
index_df = pd.read_csv(INDEX_CSV)
icpms_df = index_df[index_df['instrument'] == 'ICPMS']
sites_with_icpms_data = icpms_df['site_id'].sort_values().unique()

print(f'PM2.5 Speciation data has been measured at {len(sites_with_icpms_data)} \
sites, according to our check.')

# the difference between the counts
merged = icpms_df.merge(icpms_stations, on='site_id', how='left', indicator=True)
left_only_list = merged.loc[merged['_merge'] == 'left_only', :]

print('\nThe following sites are not reporting an ICP-MS-measured data anymore:')
not_listed = left_only_list['site_id'].sort_values().unique()
print(', '.join(map(str, not_listed)))

If you call the function `visualize_coverage_by_site_and_year` with the full name of the analyte of your interest, you can obtain a coverage of that analyte.

In [ ]:
visualize_coverage_by_site_and_year('aluminum')